In [2]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 1.1 MB/s eta 0:00:01
   -------------------- ------------------- 0.8/1.5 MB 1.5 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.4 MB/s eta 0:00:01
   --------------------------- ------------ 1.0/1.5 MB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.3/1.5 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pandas as pd

df = pd.read_csv("pakistan_weather_recommendations_10k.csv")
df

df.columns

df['']

,City Number,City Name,Latitude,Longitude,Date,Temperature (°C),Humidity (%),Precipitation (%),Wind Speed (km/h),Clothing Recommendation,Activity Recommendation,Season
0,1,Skardu,35.2971,75.6333,2023-01-01,-17.3,55,7,3.3,"Heavy winter coat, thermal layers, gloves, sca...","Skiing, indoor gatherings, hot drinks, museum ...",Winter
1,1,Skardu,35.2971,75.6333,2023-01-02,-12.4,89,2,8.9,"Heavy winter coat, thermal layers, gloves, sca...","Skiing, indoor gatherings, hot drinks, museum ...",Winter
2,1,Skardu,35.2971,75.6333,2023-01-03,-23.0,49,16,9.0,"Heavy winter coat, thermal layers, gloves, sca...","Skiing, indoor gatherings, hot drinks, museum ...",Winter
3,1,Skardu,35.2971,75.6333,2023-01-04,-13.9,89,13,3.3,"Heavy winter coat, thermal layers, gloves, sca...","Skiing, indoor gatherings, hot drinks, museum ...",Winter
4,1,Skardu,35.2971,75.6333,2023-01-05,-13.1,20,24,12.1,"Heavy winter coat, thermal layers, gloves, sca...","Skiing, indoor gatherings, hot drinks, museum ...",Winter
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,30,Okara,30.8081,73.4458,2023-11-25,10.0,37,55,1.8,"Light jacket, long sleeves, jeans, comfortable...","Hiking, outdoor dining, city tours, cycling, s...",Autumn
9996,30,Okara,30.8081,73.4458,2023-11-26,1.1,85,9,10.8,"Winter coat, sweater, warm pants, boots, light...","Winter photography, cafe hopping, light winter...",Autumn
9997,30,Okara,30.8081,73.4458,2023-11-27,6.9,25,6,5.5,"Winter coat, sweater, warm pants, boots, light...","Winter photography, cafe hopping, light winter...",Autumn
9998,30,Okara,30.8081,73.4458,2023-11-28,3.6,67,39,5.2,"Winter coat, sweater, warm pants, boots, light...","Winter photography, cafe hopping, light winter...",Autumn


In [15]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import nltk
import string

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Global stopwords set for text processing
STOPWORDS = set(stopwords.words('english'))

# -------------------------------
# HELPER FUNCTIONS
# -------------------------------

def convert_date(date_str):
    """Convert a date string into a standardized datetime object."""
    try:
        # Attempt to parse using a specified format (modify if needed)
        return pd.to_datetime(date_str, format='%Y-%m-%d')
    except Exception:
        # Let pandas infer the format, coercing errors to NaT
        return pd.to_datetime(date_str, errors='coerce')

def get_season(month):
    """Return the season as a string based on the month number."""
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

def categorize_temperature(temp):
    """
    Categorize the temperature value (in °C) into ranges.
    Change thresholds as per your domain requirements.
    """
    if temp < 0:
        return 'below_0'
    elif 0 <= temp < 10:
        return '0_10'
    elif 10 <= temp < 20:
        return '10_20'
    elif 20 <= temp < 30:
        return '20_30'
    else:
        return '30_plus'

def clean_text(text):
    """
    Lowercase, remove punctuation, tokenize text, and remove stopwords.
    Returns a list of cleaned word tokens.
    """
    # Check if text is missing or not a string
    if pd.isna(text) or not isinstance(text, str):
        return []
    text = text.lower()
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text_nopunct = text.translate(translator)
    # Tokenize text
    tokens = word_tokenize(text_nopunct)
    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in STOPWORDS]
    return filtered_tokens

# -------------------------------
# DATA PREPROCESSING PIPELINE
# -------------------------------
def preprocess_data(csv_file):
    # 1. Read CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    print("Initial data shape:", df.shape)
    
    # Columns in your dataset:
    # - 'City Number', 'City Name', 'Latitude', 'Longitude', 'Date'
    # - 'Temperature (°C)', 'Humidity (%)', 'Precipitation (%)', 'Wind Speed (km/h)'
    # - 'Clothing Recommendation', 'Activity Recommendation', 'Season'
    
    # 2. Data Cleaning
    # 2.1 Handle missing values in numerical columns via median imputation.
    numeric_cols = ['Temperature (°C)', 'Humidity (%)', 'Precipitation (%)', 'Wind Speed (km/h)']
    for col in numeric_cols:
        if col in df.columns:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
    
    # 2.2 Remove duplicate rows if any.
    df.drop_duplicates(inplace=True)
    
    # 2.3 Fix inconsistencies:
    # Convert 'Date' column to datetime format.
    if 'Date' in df.columns:
        df['Date'] = df['Date'].apply(convert_date)
    
    # Convert numeric columns to proper numeric types
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Optionally, impute missing values in 'Latitude' and 'Longitude'
    location_cols = ['Latitude', 'Longitude']
    for col in location_cols:
        if col in df.columns:
            median_val = df[col].median()
            df[col].fillna(median_val, inplace=True)
    
    # 3. Feature Engineering
    # 3.1 Temporal Features: extract month and day_of_year from 'Date'
    if 'Date' in df.columns:
        df['month'] = df['Date'].dt.month
        df['day_of_year'] = df['Date'].dt.dayofyear
        # Compute season using the month (in case you need to validate or override the provided 'Season')
        df['computed_season'] = df['month'].apply(get_season)
    
    # 3.2 Create a weather severity index by normalizing key weather parameters.
    scaler = MinMaxScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    # A simple severity index is calculated as the mean of the normalized weather values.
    df['weather_severity'] = df[numeric_cols].mean(axis=1)
    
    # 3.3 Temperature categorization.
    # For this example we're assuming scaled values can be mapped back or used for categorization.
    # If you have the raw temperature values separately, use them instead.
    # Here we multiply back by an approximate range (e.g., 40) if your temperature scale roughly follows 0-40°C.
    df['Temperature_Raw'] = df['Temperature (°C)'] * 40
    df['temp_category'] = df['Temperature_Raw'].apply(categorize_temperature)
    
    # 4. Text Processing
    # Combine the two recommendation columns into one for unified processing.
    if ('Clothing Recommendation' in df.columns) and ('Activity Recommendation' in df.columns):
        df['combined_recommendation'] = df['Clothing Recommendation'].astype(str) + " " + df['Activity Recommendation'].astype(str)
    elif 'Clothing Recommendation' in df.columns:
        df['combined_recommendation'] = df['Clothing Recommendation']
    elif 'Activity Recommendation' in df.columns:
        df['combined_recommendation'] = df['Activity Recommendation']


    print(df)
    # Clean and tokenize the combined recommendation text.
    if 'combined_recommendation' in df.columns:
        df['recommendation_tokens'] = df['combined_recommendation'].apply(clean_text)
    
    # 5. Data Splitting (Temporal Split)
    # Sort the DataFrame by Date to preserve chronological order.
    if 'Date' in df.columns:
        df.sort_values(by='Date', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    # Define train (70%), validation (15%), and test (15%) splits.
    n = len(df)
    train_end = int(n * 0.7)
    val_end = int(n * 0.85)
    
    train_df = df.iloc[:train_end].copy()
    val_df = df.iloc[train_end:val_end].copy()
    test_df = df.iloc[val_end:].copy()
    
    print("Train shape:", train_df.shape)
    print("Validation shape:", val_df.shape)
    print("Test shape:", test_df.shape)
    
    # Return the preprocessed data splits
    return train_df, val_df, test_df

# -------------------------------
# MAIN FUNCTION
# -------------------------------
if __name__ == "__main__":
    # Replace 'your_data.csv' with the actual path to your dataset.
    csv_file_path = "pakistan_weather_recommendations_10k.csv"
    train_df, val_df, test_df = preprocess_data(csv_file_path)
    
    # Optionally, save the data splits to CSV files.
    train_df.to_csv("train_processed.csv", index=False)
    val_df.to_csv("val_processed.csv", index=False)
    test_df.to_csv("test_processed.csv", index=False)
    
    print("Data preprocessing completed and processed files are saved.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\muzam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\muzam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\muzam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Initial data shape: (10000, 12)
      City Number City Name  Latitude  Longitude       Date  Temperature (°C)  \
0               1    Skardu   35.2971    75.6333 2023-01-01          0.114458   
1               1    Skardu   35.2971    75.6333 2023-01-02          0.212851   
2               1    Skardu   35.2971    75.6333 2023-01-03          0.000000   
3               1    Skardu   35.2971    75.6333 2023-01-04          0.182731   
4               1    Skardu   35.2971    75.6333 2023-01-05          0.198795   
...           ...       ...       ...        ...        ...               ...   
9995           30     Okara   30.8081    73.4458 2023-11-25          0.662651   
9996           30     Okara   30.8081    73.4458 2023-11-26          0.483936   
9997           30     Okara   30.8081    73.4458 2023-11-27          0.600402   
9998           30     Okara   30.8081    73.4458 2023-11-28          0.534137   
9999           30     Okara   30.8081    73.4458 2023-11-29          0.632530